Ce notebook n'est qu'un prototype.
Le code définitif doit être écrit dans le fichier main.py

In [1]:
import vtk
import itk

In [2]:
# Ajouter ici les import nécessaires autres que vtk et itk
# import ...

In [20]:
# Les chemins relatif qui mènent au data
IMAGE1_PATH = "./Data/case6_gre1.nrrd"
IMAGE2_PATH = "./Data/case6_gre2.nrrd"

# Chargement des images

In [21]:
reader = vtk.vtkNrrdReader()
reader.SetFileName(IMAGE1_PATH)
reader.Update()
image1 = reader.GetOutput()

reader.SetFileName(IMAGE2_PATH)
reader.Update()
image2 = reader.GetOutput()

In [22]:
def visualize_image(image, title="Image"):
    extent = image.GetExtent()
    middle_slice = (extent[4] + extent[5]) // 2

    viewer = vtk.vtkImageViewer2()
    viewer.SetInputData(image)
    viewer.GetRenderer().SetBackground(0.1, 0.1, 0.1)
    viewer.SetSlice(middle_slice)
    viewer.SetColorWindow(255)
    viewer.SetColorLevel(128)
    viewer.Render()
    viewer.GetRenderWindow().SetWindowName(title)

    interactor = vtk.vtkRenderWindowInteractor()
    viewer.SetupInteractor(interactor)
    interactor.Initialize()
    interactor.Start()



In [23]:
visualize_image(image1, title="Image 1")
visualize_image(image2, title="Image 2")

In [17]:
def rigid_registration(fixed_path, moving_path, output_path=None):
    fixed = itk.imread(fixed_path, itk.F)
    moving = itk.imread(moving_path, itk.F)

    TransformType = itk.BSplineTransform[itk.D, 3, 3]
    initial_transform = TransformType.New()

    MetricType = itk.MattesMutualInformationImageToImageMetricv4[
        itk.Image[itk.F, 3], itk.Image[itk.F, 3]
    ]
    metric = MetricType.New()
    metric.SetNumberOfHistogramBins(50)

    OptimizerType = itk.RegularStepGradientDescentOptimizerv4[itk.D]
    optimizer = OptimizerType.New()
    optimizer.SetLearningRate(4.0)
    optimizer.SetMinimumStepLength(0.001)
    optimizer.SetNumberOfIterations(200)

    RegistrationType = itk.ImageRegistrationMethodv4[
        itk.Image[itk.F, 3], itk.Image[itk.F, 3]
    ]
    registration = RegistrationType.New()
    registration.SetFixedImage(fixed)
    registration.SetMovingImage(moving)
    registration.SetInitialTransform(initial_transform)
    registration.SetMetric(metric)
    registration.SetOptimizer(optimizer)
    registration.SetShrinkFactorsPerLevel([4,2,1])
    registration.SetSmoothingSigmasPerLevel([2,1,0])

    registration.Update()
    final_transform = registration.GetTransform()

    ResampleFilterType = itk.ResampleImageFilter[
        itk.Image[itk.F,3], itk.Image[itk.F,3]
    ]
    resampler = ResampleFilterType.New()
    resampler.SetInput(moving)
    resampler.SetTransform(final_transform)
    resampler.SetUseReferenceImage(True)
    resampler.SetReferenceImage(fixed)
    resampler.SetInterpolator(
        itk.LinearInterpolateImageFunction[
            itk.Image[itk.F,3], itk.D
        ].New()
    )
    resampler.Update()

    if output_path:
        itk.imwrite(resampler.GetOutput(), output_path)
    return final_transform, resampler.GetOutput()

In [ ]:
IMAGE_RECALEE_PATH = "Data/case6_gre2_rigid.nrrd"

transform, moved = rigid_registration(
    "Data/case6_gre1.nrrd",
    "Data/case6_gre2.nrrd",
    output_path=IMAGE_RECALEE_PATH
)

itk.imwrite(moved, IMAGE_RECALEE_PATH)

In [ ]:
reader.SetFileName(IMAGE_RECALEE_PATH)
reader.Update()
image_recalee = reader.GetOutput()
visualize_image(image_recalee, "GRE2 recalée rigide")
visualize_image(image1, "Image 1")
visualize_image(image2, "Image 2")

# Segmentation de la tumeur

# Analyse et visualisation des changements